# Testing Environment

In [1]:
print("hello")

hello


In [2]:
import torch
print("pytorch")

x = torch.rand(5, 3)
print(x)

pytorch
tensor([[0.8646, 0.7191, 0.3828],
        [0.3790, 0.3049, 0.8329],
        [0.7769, 0.3626, 0.5144],
        [0.2469, 0.7711, 0.4298],
        [0.1972, 0.9742, 0.5497]])


In [3]:
torch.cuda.is_available()

True

# 0

In [5]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%
100.0%
100.0%
100.0%


In [7]:
batch_size = 64


# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break



Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

model.train()

Epoch 1
-------------------------------
loss: 1.176637  [   64/60000]
loss: 1.166381  [ 6464/60000]
loss: 0.986274  [12864/60000]
loss: 1.114092  [19264/60000]
loss: 1.012665  [25664/60000]
loss: 1.027703  [32064/60000]
loss: 1.060163  [38464/60000]
loss: 1.004546  [44864/60000]
loss: 1.047902  [51264/60000]
loss: 0.969414  [57664/60000]
Test Error: 
 Accuracy: 65.5%, Avg loss: 0.990255 

Epoch 2
-------------------------------
loss: 1.051432  [   64/60000]
loss: 1.062118  [ 6464/60000]
loss: 0.865741  [12864/60000]
loss: 1.016534  [19264/60000]
loss: 0.920640  [25664/60000]
loss: 0.930006  [32064/60000]
loss: 0.978278  [38464/60000]
loss: 0.927226  [44864/60000]
loss: 0.966601  [51264/60000]
loss: 0.898757  [57664/60000]
Test Error: 
 Accuracy: 66.8%, Avg loss: 0.914499 

Epoch 3
-------------------------------
loss: 0.959666  [   64/60000]
loss: 0.990522  [ 6464/60000]
loss: 0.780647  [12864/60000]
loss: 0.948453  [19264/60000]
loss: 0.859986  [25664/60000]
loss: 0.859316  [32064/600

#### Saving and using models

In [15]:
torch.save(model.state_dict(), "models/modelZero.pth")
print("Saved PyTorch Model State to models/modelZero.pth")

Saved PyTorch Model State to models/modelZero.pth


In [19]:
modeltwo = NeuralNetwork().to(device)
modeltwo.load_state_dict(torch.load("models/modelZero.pth", weights_only=True))

<All keys matched successfully>

#### using a class to make predictions

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Tensors

In [1]:
import torch
import numpy as np

In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
print(x_data)
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [7]:
x_ones = torch.ones_like(x_data)
x_rand = torch.rand_like(x_data, dtype=torch.float)
print(x_ones)
print(x_rand)

tensor([[1, 1],
        [1, 1]])
tensor([[0.7254, 0.5776],
        [0.6088, 0.0166]])


In [8]:
shape = (2,3,)
zeros_tensor = torch.zeros(shape)
zeros_tensor

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [9]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [ ]:
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())